In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
%matplotlib inline

In [ ]:
df = pd.read_csv(r'/kaggle/input/univai-dataset/univ.ai_Training Data.csv')

In [ ]:
df.head()

In [ ]:
cat_cols = [col for col in df.columns if df[col].dtype == 'object']

In [ ]:
df[cat_cols].nunique().sort_values()

In [ ]:
df.drop(columns=['state', 'profession','city'], axis=1, inplace=True)

In [ ]:
cat_cols.remove('profession')

In [ ]:
cat_cols.remove('city')
cat_cols.remove('state')

In [ ]:
cat_cols

In [ ]:
from sklearn.preprocessing import OneHotEncoder
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)

In [ ]:
df1 = pd.DataFrame(OH_encoder.fit_transform(df[cat_cols]))

In [ ]:
df1.index = df.index

In [ ]:
df = pd.concat([df,df1],axis=1)

In [ ]:
df.drop(columns=['married', 'house_ownership','car_ownership'], axis=1, inplace=True)

In [ ]:
df.head()

In [ ]:
df_test = pd.read_csv(r'/kaggle/input/univai-dataset/univ.ai_Training Data.csv')

In [ ]:
df_test.rename(columns={'id':'Id'}, inplace=True)

In [ ]:
cat_cols = [col for col in df_test.columns if df_test[col].dtype == 'object']

In [ ]:
df_test[cat_cols].nunique().sort_values()

In [ ]:
df_test.drop(columns=['state', 'profession','city'], axis=1, inplace=True)

In [ ]:
cat_cols.remove('profession')
cat_cols.remove('city')
cat_cols.remove('state')

In [ ]:
cat_cols

In [ ]:
df2 = pd.DataFrame(OH_encoder.fit_transform(df_test[cat_cols]))

In [ ]:
df2.index = df_test.index

In [ ]:
df_test = pd.concat([df_test,df2],axis=1)

In [ ]:
df_test.head()

In [ ]:
df_test.drop(columns=['married', 'house_ownership','car_ownership'], axis=1, inplace=True)

In [ ]:
df_test.head()

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
model_KNN = KNeighborsClassifier()

In [ ]:
X = df[['Id','income','age','experience','current_job_years','current_house_years']]
y = df['risk_flag']

In [ ]:
X_test = df_test[['Id','income','age','experience','current_job_years','current_house_years']]

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [ ]:
X = scaler.fit_transform(X)

In [ ]:
X_test = scaler.transform(X_test)

In [ ]:
leaf_size = list(range(1,50))
n_neighbors = list(range(1,30))
p=[1,2]
#convert to dictionary
hyperparameters = dict(leaf_size=leaf_size, n_neighbors=n_neighbors, p=p)
#Making model
clf = GridSearchCV(model_KNN, hyperparameters, cv=10)
best_model = clf.fit(X,y)
#Best Hyperparameters Value
print('Best leaf_size:', best_model.best_estimator_.get_params()['leaf_size'])
print('Best p:', best_model.best_estimator_.get_params()['p'])
print('Best n_neighbors:', best_model.best_estimator_.get_params()['n_neighbors'])

In [ ]:
pred_SVM = best_model.predict(X_test)

In [ ]:
final_data = {'id': df_test.Id, 'risk_flag': pred_SVM}

In [ ]:
sub = pd.DataFrame(final_data)

In [ ]:
sub.head()

In [ ]:
filename= 'submission_final_KNN.csv'
sub.to_csv(filename, index=False)